In [ ]:
#movie rec using collaborative data and matrix factorization
import csv
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from scipy import optimize
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_validation import train_test_split

#some code from http://alexabate.github.io/2016/11/05/movie-lens.html
#some code from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/

user_item_ratings = pd.read_csv('ratings.csv', sep=',')
#user_item_ratings.head()
number_of_unique_users = len(user_item_ratings['userId'].unique())
number_of_unique_movies = len(user_item_ratings['movieId'].unique())
number_of_ratings = len(user_item_ratings)
#print user_item_ratings
print number_of_unique_users
print number_of_unique_movies

ratingsMatrix=pd.pivot_table(user_item_ratings, values='rating', fill_value=0, columns='userId', index='movieId')#
#print ratingsMatrix.head

ratingsMatrix_test=ratingsMatrix.iloc[6044:,:]

ratingsMatrix=ratingsMatrix.iloc[:6043,:]


did_rate = (ratingsMatrix != 0) * 1
did_rate_test=(ratingsMatrix_test != 0) * 1

#n_movies=number_of_unique_movies
n_movies=ratingsMatrix.shape[0]
n_users=number_of_unique_users


#normalize_ratings from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]
    ratings_mean = np.zeros(shape = (num_movies, 1))
    ratings_norm = np.zeros(shape = ratings.shape)
    
    for i in range(0, num_movies):
        # Get all the indexes where there is a 1
        idx = np.where(did_rate.iloc[i,:] ==1)[0]
        # Calculate mean rating of ith movie only from users that gave a rating
        ratings_mean[i] = np.mean(ratings.iloc[i, idx])
         #ratings_mean[i]
        ratings_norm[i, idx] = ratings.iloc[i, idx] - ratings_mean[i]
        #(EKC: make sure these indices aren't off by one)
        #print i
    return (ratings_norm, ratings_mean)

ratings_norm, ratings_mean = normalize_ratings(ratingsMatrix, did_rate)

did_rate=did_rate.as_matrix()

svd = TruncatedSVD(n_components=3, n_iter=10, random_state=42)
X_tr = svd.fit_transform(ratings_norm)
print X_tr.shape

X_o = svd.inverse_transform(X_tr)
#that would be the predicted ratings
difference = X_o*did_rate-ratings_norm
cost = np.sum((difference)**2)/2
print cost

movie_info = pd.read_csv('movies.csv', sep=',')



#need a list of all the genres--basically just go through the rows and add whichever ones are unique
all_genres=set()
for row in range(0,movie_info.shape[0]):
    genres=movie_info['genres'][row]
    genres_split=genres.split('|')
    all_genres.update(genres_split)
print all_genres
all_genres=list(all_genres)
print all_genres[0]
    
n_features=len(all_genres)
 
n_movies_genres=len(movie_info['movieId'].unique())
genres_in_descriptions=pd.DataFrame(np.zeros([n_movies_genres, n_features]))
print genres_in_descriptions.shape
#just want to combine the movies
print set(movie_info['movieId'].unique()) - set(user_item_ratings['movieId'].unique())
print set(user_item_ratings['movieId'].unique()) - set(movie_info['movieId'].unique())
#some movies have genres but haven't been rated. ignoring them for now.
# DELETE UNRATED MOVIES
#this is slow but ony has to happen once
n_movies=len(user_item_ratings['movieId'].unique())
genresMatrix=np.zeros([n_movies, n_features])
#print ratings_norm.shape
for ii in range(0,n_movies):
    thisMovie=ratingsMatrix.index.values[ii]
    Index=np.where(movie_info['movieId']==thisMovie)
    genres=movie_info['genres'][Index[0]]
    #genres_split=genres.split('|')
    gInd=0
    for g in all_genres:
        if g in str(genres):
            genresMatrix[ii, gInd]=1
        gInd=gInd+1
    
    
    #convert ratings to probability of liking movie
# first z-score them (express each rating in terms of the standard deviation of that user's ratings)
stddevs=np.std(ratings_norm,axis=0)
ratings_zscored=ratings_norm/(stddevs)
# then get the probability (note that this assumes the ratings are distributed normally)
from scipy.stats import norm
probs=norm.cdf(ratings_zscored)

#bayesian model

ProbabilityOfLikingMovies=probs[1:,:].T # 
wordsInAllDescriptions=genresMatrix[1:,:]
#(leave one out for testing--later decide on the best cross-validation method)

#log likelihood of a user liking a movie is the sum of P(like) for all movies containing each word in the movie's description
# movie's description=genre (and tags? but user would not have tagged a new movie. Can include all tags for each movie?)
weightedP=ProbabilityOfLikingMovies.dot(wordsInAllDescriptions) #
                #(1-by-movies)         *  (movies-by-featurewords)
                #(users-by-movies)         *  (movies-by-featurewords)
            
wordsInMovieDescription=genresMatrix[0,:]
    
logPLikeMovie=weightedP.dot(wordsInMovieDescription)
#(users-by-features * features-by-1?)
#sum the probability of liking each movie that contains each word--this is a matrix multiplication, right?


# do this for every movie
logPAllMovies=np.zeros(ratingsMatrix.T.shape)
for ii in range(0,n_movies):
    ProbabilityOfLikingMovies=np.delete(probs, ii, 0).T # 
    wordsInAllDescriptions=np.delete(genresMatrix, ii, 0)
    #(leave one out for testing--later decide on the best cross-validation method)

    #log likelihood of a user liking a movie is the sum of P(like) for all movies containing each word in the movie's description
    # movie's description=genre (and tags? but user would not have tagged a new movie. Can include all tags for each movie?)
    weightedP=ProbabilityOfLikingMovies.dot(wordsInAllDescriptions) #
                    #(1-by-movies)         *  (movies-by-featurewords)
                    #(users-by-movies)         *  (movies-by-featurewords)
            
    wordsInMovieDescription=genresMatrix[ii,:]
    
    logPLikeMovie=weightedP.dot(wordsInMovieDescription)
    #(users-by-features * features-by-1?)
    
    logPAllMovies[:,ii]=logPLikeMovie
    
    #sum the probability of liking each movie that contains each word--this is a matrix multiplication, right?
    if ii % 50 == 0:
        print ii

    

    
#convert the log likelihoods to z scores and then to ratings--fill in the ratings matrix 
predictedRatingsNorm=logPAllMovies.T
PRNmeans=np.mean(predictedRatingsNorm, axis=0)
PRNstddevs=np.std(predictedRatingsNorm,axis=0)

predictedRatingsNorm=(predictedRatingsNorm-PRNmeans)/(PRNstddevs)
#scale by the standard rating deviation
predictedRatingsNorm=predictedRatingsNorm * stddevs #(make sure that's the right way to actually do this multiplication)




In [1]:
# fill in missing data in ratingsNorm with predictedRatingsNorm, then do the matrix factorization

denseMatrix=np.empty_like(predictedRatingsNorm) 
denseMatrix[:]=predictedRatingsNorm
Inds=np.where(did_rate==1)
denseMatrix[Inds]=ratings_norm[Inds]

# then do the collab filter
svd = TruncatedSVD(n_components=3, n_iter=10, random_state=42)
finalPredictedRatings = svd.fit_transform(denseMatrix)

#how well does it do? test it on the last third of the data
 

NameError: name 'pedictedRatingsNorm' is not defined